In [16]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd

base_dir = "/content/drive/MyDrive/JeweleryRecognition"
image_dir = os.path.join(base_dir, "Images")
csv_path = os.path.join(base_dir, "Labels.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install faiss-cpu


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-a2mp4h2g
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-a2mp4h2g
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [18]:
import pandas as pd
from PIL import Image
import os

df = pd.read_csv(csv_path)
df['filepath'] = df['FILENAME'].apply(lambda x: os.path.join(image_dir, x))
df = df[df['filepath'].apply(os.path.exists)].reset_index(drop=True)


In [19]:
import torch
import clip
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def extract_features(image_paths):
    features = []
    for path in tqdm(image_paths):
        image = preprocess(Image.open(path).convert("RGB")).unsqueeze(0).to(device)
        with torch.no_grad():
            feature = model.encode_image(image).cpu().numpy()
        features.append(feature[0])
    return features

image_features = extract_features(df['filepath'].tolist())


100%|██████████| 24/24 [00:15<00:00,  1.59it/s]


In [20]:
import faiss
import numpy as np

features_np = np.array(image_features).astype("float32")
faiss.normalize_L2(features_np)

index = faiss.IndexFlatIP(features_np.shape[1])
index.add(features_np)


In [21]:
!pip install gradio

In [22]:
import gradio as gr
import matplotlib.pyplot as plt
import io
import math
from PIL import Image
import torch

stored_state = {"query_image": None}

def run_search(input_image):
    if input_image is None:
        return gr.update(value="❗ Please upload an image."), None, gr.update(visible=True), gr.update(visible=False)

    if not is_jewellery(input_image):
        return gr.update(value="🚫 That doesn’t look like jewellery. Try another image."), None, gr.update(visible=True), gr.update(visible=False)

    stored_state["query_image"] = input_image
    img_t = preprocess(input_image.convert("RGB")).unsqueeze(0).to(device)
    with torch.no_grad():
        feats = model.encode_image(img_t).cpu().numpy().astype("float32")
    faiss.normalize_L2(feats)
    D, I = index.search(feats, k=5)
    idxs = [i for i, d in zip(I[0], D[0]) if d >= 0.25]
    if not idxs:
        return gr.update(value="🔍 No matches found."), None, gr.update(visible=False), gr.update(visible=True)

    df_sim = df.iloc[idxs]
    imgs = [input_image] + [Image.open(p) for p in df_sim['filepath']]
    caps = ["Query Image"] + [f"{r['FILENAME']}\n{r['JEWELLERY_TYPE']} | {r['METAL_USED']}" for _, r in df_sim.iterrows()]
    cols, rows = 3, math.ceil(len(imgs) / 3)
    fig, axs = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
    axs = axs.flatten()
    for ax in axs:
        ax.axis("off")
    for i, im in enumerate(imgs):
        axs[i].imshow(im)
        axs[i].set_title(caps[i], fontsize=10)
    plt.tight_layout()
    buf = io.BytesIO()
    fig.savefig(buf, format="png")
    plt.close(fig)
    buf.seek(0)
    result_img = Image.open(buf)
    return gr.update(value="✅ Matches found!"), result_img, gr.update(visible=False), gr.update(visible=True)

def go_back():
    stored_state["query_image"] = None
    return None, "", gr.update(visible=True), gr.update(visible=False)

with gr.Blocks(title="JewelVision") as demo:
    gr.Markdown("""
<link href="https://fonts.googleapis.com/css2?family=Playfair+Display:wght@700&family=Roboto:wght@400&display=swap" rel="stylesheet">
<div class="navbar">
    <div class="nav-logo">🔶 JewelVision</div>
    <div class="nav-links">
        <a href="#">Home</a>
        <a href="#">About</a>
        <a href="#">Upload</a>
        <a href="#">Contact</a>
    </div>
</div>

<div class="main-header fade-in">
    <h1>JewelVision</h1>
    <p>Discover similar jewellery styles with AI-powered image search</p>
</div>
""")

    with gr.Column(visible=True, elem_id="search-col") as search_col:
        gr.Markdown("<h3 style='text-align:center;'>Upload a Jewellery Image 💍</h3>")
        image_in = gr.Image(type="pil", label="", width=400)
        status_txt = gr.Textbox(
            label="", interactive=False, lines=1,
            value="Please upload a jewellery image.",
            show_label=False,
            elem_classes="status-txt"
        )

    with gr.Column(visible=False, elem_id="results-col") as results_col:
        gr.Markdown("<h3 style='text-align:center;'>🎯 Similar Matches</h3>")
        output_img = gr.Image(show_label=False, width=1000)
        back_btn = gr.Button("⬅️ Back to Search", elem_classes="back-button")

    image_in.change(fn=run_search, inputs=[image_in],
                    outputs=[status_txt, output_img, search_col, results_col])
    back_btn.click(fn=go_back, inputs=[],
                   outputs=[image_in, status_txt, search_col, results_col])

    demo.css = """
.navbar {
    display: flex;
    justify-content: space-between;
    align-items: center;
    background-color: #f8f1e4;
    padding: 15px 30px;
    border-bottom: 2px solid #e0d2b6;
    font-family: 'Roboto', sans-serif;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    position: sticky;
    top: 0;
    z-index: 10;
}
.nav-logo {
    font-size: 1.6em;
    font-weight: bold;
    color: #d4af37;
}
.nav-links a {
    margin-left: 25px;
    text-decoration: none;
    color: #4B3B2B;
    font-weight: 500;
    transition: color 0.3s ease, transform 0.3s ease;
}
.nav-links a:hover {
    color: #d4af37;
    transform: translateY(-2px);
}
.main-header {
    text-align: center;
    padding: 30px;
    animation: fadeInUp 1s ease-in-out;
}
.main-header h1 {
    font-family: 'Playfair Display', serif;
    font-size: 4em;
    color: #d4af37;
    margin: 0;
}
.main-header p {
    font-size: 1.5em;
    color: #4B3B2B;
    font-family: 'Roboto', sans-serif;
}
#search-col, #results-col {
    align-items: center;
    padding: 25px;
    background: #fff8f0;
    border-radius: 20px;
    box-shadow: 0 0 20px rgba(0,0,0,0.1);
}
.back-button {
    background: #8B4513;
    color: white;
    font-weight: bold;
    padding: 12px 25px;
    border-radius: 10px;
    margin-top: 15px;
}
.back-button:hover {
    background: #A0522D;
}
.status-txt textarea {
    text-align: center !important;
    font-weight: 600;
    font-size: 1.2em;
    color: #4B3B2B;
}
body, .gradio-container, h1, h3, p, label {
    text-align: center !important;
    font-family: 'Roboto', sans-serif;
}
input[type="file"] {
    margin-top: 15px;
    text-align: center;
}
@keyframes fadeInUp {
    from {
        opacity: 0;
        transform: translateY(30px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}
.fade-in {
    animation: fadeInUp 1s ease-in-out;
}
"""

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5009e74860a8d9feec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
